In [2]:
from tkinter import *
import tkinter as tk
import math
from tkinter import filedialog
import os
import numpy as np
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True
import imutils
from PIL import Image
from PIL import ImageTk
# global variables
bg = None
import time
from PIL import ImageTk, Image
from resizeimage import resizeimage
import cv2
global jk
jk=0
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import librosa
from scipy.stats import kurtosis
from scipy.stats import skew

def extract_features(y,sr=22050,n_fft=1024,hop_length=512):
    features = {'centroid': librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel(),
                'flux': librosa.onset.onset_strength(y=y, sr=sr).ravel(),
                'rmse': librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel(),
                'zcr': librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel(),
                'contrast': librosa.feature.spectral_contrast(y, sr=sr).ravel(),
                'bandwidth': librosa.feature.spectral_bandwidth(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel(),
                'flatness': librosa.feature.spectral_flatness(y, n_fft=n_fft, hop_length=hop_length).ravel(),
                'rolloff': librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()}

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, n_fft=n_fft, hop_length=hop_length, n_mfcc=20)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()

    # Get statistics from the vectors
    def get_feature_stats(features):
        result = {}
        for k, v in features.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result

    dict_agg_features = get_feature_stats(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y=y,sr=sr,hop_length=hop_length)[0]

    return dict_agg_features

def callback(selection):
    global xname
    xname=selection

    
class Window(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)                 
        self.master = master

        # changing the title of our master widget      
        self.master.title("Project")
        
        self.pack(fill=BOTH, expand=1)
        w = tk.Label(root, 
		 text="Music Genre Classification and Recommendation ",
		 fg = "light green",
		 bg = "dark green",
		 font = "Helvetica 20 bold italic")
        w.pack()
        w.place(x=200, y=0)
        # creating a button instance
        quitButton = Button(self,command=self.start, text="Upload music",bg="yellow",activebackground="dark red",width=20)
        quitButton.place(x=10, y=150)
        quitButton = Button(self,command=self.process,text="Predict genere",bg="yellow",activebackground="dark red",width=20)
        quitButton.place(x=10, y=300)
        #quitButton = Button(self,command=self.stopprocess, text="Stop Process",fg="blue",activebackground="dark red",width=20)
        #quitButton.place(x=10, y=300)

        load = Image.open("logo.png")
        render = ImageTk.PhotoImage(load)

        image2=Label(self, image=render,borderwidth=5, highlightthickness=5, height=500, width=500, bg='grey')
        image2.image = render
        image2.place(x=180, y=70)

        
#       2nd row
        contents ="  Waiting for Results..."
        global T
        T = Text(self, height=19, width=25)
        T.pack()
        T.place(x=700, y=50)
        T.insert(END,contents)
        print(contents)
        
#       3rd row
        #image5.place(x=300, y=490)

#       Functions

    def start(self, event=None):
        contents ="Loading Image..."
        global T,filename
        T = Text(self, height=19, width=25)
        #T.pack()
        T.place(x=700, y=50)
        T.insert(END,contents)
        print(contents)
        global cap,jk
        from tkinter import filedialog
        filename = filedialog.askopenfilename(title='open')
        #self.from_array = Image.fromarray(frame)
        image = cv2.imread("logo.png")
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        # org
        org = (150, 220)
        # fontScale
        fontScale = 1
        # Blue color in BGR
        color = (255, 0, 0)
        # Line thickness of 2 px
        thickness = 1 
        # Using cv2.putText() method
        image = cv2.putText(image, 'Music Loaded', org, font, 
                           fontScale, color, thickness, cv2.LINE_AA)

        self.from_array = Image.fromarray(image)
        render = ImageTk.PhotoImage(self.from_array)
        image2=Label(self, image=render,borderwidth=5, highlightthickness=5, height=500, width=500, bg='red')
        image2.image = render
        image2.place(x=180, y=70)
        
        contents="Music Loaded successfully !!"
        
        T = Text(self, height=19, width=25)
        #T.pack()
        T.place(x=700, y=50)
        T.insert(END,contents)
        print(contents)
        
    def close_window(): 
        Window.destroy()
        
    def process(self, event=None):
        global T,rep,rep1,filename
        global xname
        global data
        contents="Loading Massage ..."
        T = Text(self, height=19, width=25)
        #T.pack()
        import pandas as pd
        T.place(x=700, y=50)
        T.insert(END,contents)
        arr_features=[]
        y, sr = librosa.load(filename, duration=30)
        dict_features=extract_features(y=y,sr=sr)
        arr_features.append(dict_features)
        df=pd.DataFrame(data=arr_features)
        print(df.head())
        print(df.shape)
        df.to_csv('test_data1.csv',index=False)

        df=pd.read_csv('test_data1.csv')

        X=df#df.drop('label',axis=1).values

        df=pd.read_csv('train_data.csv')
        df_t=pd.read_csv('train_data.csv')




        X_train=df.drop('label',axis=1).values
        y_train=df['label'].values

        X_test=df_t.drop('label',axis=1).values
        y_test=df_t['label'].values



        
        from sklearn.ensemble import RandomForestClassifier

        clfRF = RandomForestClassifier(max_depth=2, random_state=0)
        clfRF.fit(X_train, y_train)
        preds = clfRF.predict(X)

        genres = ['blues' ,'classical',' country', 'disco', 'hiphop' ,'jazz',' metal',' pop',' reggae rock']
        print('Predicted genres is ='+genres [preds[0]])

        image = cv2.imread("logo.png")
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        # org
        org = (100, 220)
        # fontScale
        fontScale = .6
        # Blue color in BGR
        color = (255, 0, 0)
        # Line thickness of 2 px
        thickness = 1 
        # Using cv2.putText() method
        image = cv2.putText(image,'Predicted genres='+genres [preds[0]] , org, font, 
                           fontScale, color, thickness, cv2.LINE_AA)

        self.from_array = Image.fromarray(image)
        render = ImageTk.PhotoImage(self.from_array)
        image2=Label(self, image=render,borderwidth=5, highlightthickness=5, height=500, width=500, bg='orange')
        image2.image = render
        image2.place(x=180, y=70)
        T = Text(self, height=19, width=25)
        #T.pack()
        T.place(x=700, y=50)
        T.insert(END,contents)
        print(contents)
            
       
    def stop(self, event=None):
        
        global T,cap
        
        cap.release()
        T = Text(self, height=20, width=25)
        #T.pack()
        T.place(x=860, y=50)
        T.insert(END,'Camera closed successfully')
        


     
root = Tk()
root.geometry("900x450")
app = Window(root)
root.mainloop()


  Waiting for Results...
